# Voting Ensemble with Enhanced Features

**Goal:** Reduce CV-LB gap (currently +0.0732) by:
1. Using diverse models in soft voting ensemble
2. Simpler hyperparameters to reduce overfitting
3. Adding robust features: Deck, FamilySize_Bin, Age_Bin

**Reference:** Strategy recommends simpler models to improve generalization

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (RandomForestClassifier, GradientBoostingClassifier, 
                              ExtraTreesClassifier, AdaBoostClassifier, VotingClassifier)
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

# Load data
train = pd.read_csv('/home/data/train.csv')
test = pd.read_csv('/home/data/test.csv')

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")

Train shape: (891, 12)
Test shape: (418, 11)


In [2]:
import re

def extract_title(name):
    """Extract title from name using regex"""
    title_search = re.search(r' ([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1)
    return ""

def process_features(df, is_train=True):
    """Process features for train/test data with enhanced features"""
    df = df.copy()
    
    # 1. Title extraction
    df['Title'] = df['Name'].apply(extract_title)
    df['Title'] = df['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don', 
                                        'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    df['Title'] = df['Title'].replace('Mlle', 'Miss')
    df['Title'] = df['Title'].replace('Ms', 'Miss')
    df['Title'] = df['Title'].replace('Mme', 'Mrs')
    
    # 2. Family features
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    df['IsAlone'] = (df['FamilySize'] == 1).astype(int)
    
    # 3. FamilySize_Bin (NEW - captures non-linear relationship)
    df['FamilySize_Bin'] = pd.cut(df['FamilySize'], bins=[0, 1, 4, 11], labels=[0, 1, 2]).astype(int)
    # 0 = alone, 1 = small (2-4), 2 = large (5+)
    
    # 4. Has_Cabin feature
    df['Has_Cabin'] = df['Cabin'].notna().astype(int)
    
    # 5. Deck feature (NEW - strong signal from analysis)
    df['Deck'] = df['Cabin'].apply(lambda x: x[0] if pd.notna(x) else 'U')
    
    # 6. Embarked - fill missing with mode
    df['Embarked'] = df['Embarked'].fillna('S')
    
    # 7. Fare - fill missing with median by Pclass
    if df['Fare'].isna().any():
        df['Fare'] = df.groupby('Pclass')['Fare'].transform(lambda x: x.fillna(x.median()))
    
    return df

# Process train and test
train_processed = process_features(train, is_train=True)
test_processed = process_features(test, is_train=False)

print("Title distribution:")
print(train_processed['Title'].value_counts())
print("\nDeck distribution:")
print(train_processed['Deck'].value_counts())
print("\nFamilySize_Bin distribution:")
print(train_processed['FamilySize_Bin'].value_counts())

Title distribution:
Title
Mr        517
Miss      185
Mrs       126
Master     40
Rare       23
Name: count, dtype: int64

Deck distribution:
Deck
U    687
C     59
B     47
D     33
E     32
A     15
F     13
G      4
T      1
Name: count, dtype: int64

FamilySize_Bin distribution:
FamilySize_Bin
0    537
1    292
2     62
Name: count, dtype: int64


In [3]:
# Age imputation using median by Pclass, Sex, Title (from training data only)
def impute_age(train_df, test_df):
    """Impute age using median by Pclass, Sex, Title"""
    train_df = train_df.copy()
    test_df = test_df.copy()
    
    # Calculate medians from training data
    age_medians = train_df.groupby(['Pclass', 'Sex', 'Title'])['Age'].median()
    
    def get_median_age(row, medians, fallback_median):
        if pd.isna(row['Age']):
            try:
                return medians.loc[(row['Pclass'], row['Sex'], row['Title'])]
            except KeyError:
                return fallback_median
        return row['Age']
    
    fallback = train_df['Age'].median()
    train_df['Age'] = train_df.apply(lambda x: get_median_age(x, age_medians, fallback), axis=1)
    test_df['Age'] = test_df.apply(lambda x: get_median_age(x, age_medians, fallback), axis=1)
    
    return train_df, test_df

train_processed, test_processed = impute_age(train_processed, test_processed)

# Age_Bin feature (NEW - captures child survival advantage)
def add_age_bin(df):
    df = df.copy()
    df['Age_Bin'] = pd.cut(df['Age'], bins=[0, 16, 32, 48, 100], labels=[0, 1, 2, 3]).astype(int)
    # 0 = Child (0-16), 1 = Young (16-32), 2 = Middle (32-48), 3 = Senior (48+)
    return df

train_processed = add_age_bin(train_processed)
test_processed = add_age_bin(test_processed)

print(f"Missing values after imputation:")
print(f"Train Age missing: {train_processed['Age'].isna().sum()}")
print(f"Test Age missing: {test_processed['Age'].isna().sum()}")
print("\nAge_Bin distribution:")
print(train_processed['Age_Bin'].value_counts().sort_index())

Missing values after imputation:
Train Age missing: 0
Test Age missing: 0

Age_Bin distribution:
Age_Bin
0    104
1    490
2    216
3     81
Name: count, dtype: int64


In [4]:
# Encode categorical features
def encode_features(train_df, test_df):
    """Encode categorical features"""
    train_df = train_df.copy()
    test_df = test_df.copy()
    
    # Sex encoding
    train_df['Sex'] = train_df['Sex'].map({'female': 0, 'male': 1})
    test_df['Sex'] = test_df['Sex'].map({'female': 0, 'male': 1})
    
    # Embarked encoding
    embarked_map = {'S': 0, 'C': 1, 'Q': 2}
    train_df['Embarked'] = train_df['Embarked'].map(embarked_map)
    test_df['Embarked'] = test_df['Embarked'].map(embarked_map)
    
    # Title encoding
    title_map = {'Mr': 1, 'Miss': 2, 'Mrs': 3, 'Master': 4, 'Rare': 5}
    train_df['Title'] = train_df['Title'].map(title_map)
    test_df['Title'] = test_df['Title'].map(title_map)
    
    # Deck encoding (U=0, A-G=1-7, T=8)
    deck_map = {'U': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'T': 8}
    train_df['Deck'] = train_df['Deck'].map(deck_map)
    test_df['Deck'] = test_df['Deck'].map(deck_map)
    
    return train_df, test_df

train_encoded, test_encoded = encode_features(train_processed, test_processed)

# Select features for modeling (enhanced feature set)
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 
            'Title', 'FamilySize', 'IsAlone', 'Has_Cabin', 
            'Deck', 'FamilySize_Bin', 'Age_Bin']  # NEW features

X = train_encoded[features].values
y = train_encoded['Survived'].values
X_test = test_encoded[features].values

print(f"Feature matrix shape: {X.shape}")
print(f"Test matrix shape: {X_test.shape}")
print(f"\nFeatures ({len(features)}): {features}")

Feature matrix shape: (891, 14)
Test matrix shape: (418, 14)

Features (14): ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Title', 'FamilySize', 'IsAlone', 'Has_Cabin', 'Deck', 'FamilySize_Bin', 'Age_Bin']


In [5]:
# Define voting ensemble with simpler hyperparameters (to reduce overfitting)
# Following strategy: use diverse models with regularization

models = [
    ('lr', LogisticRegression(C=0.1, max_iter=1000, random_state=42)),  # Simple, regularized
    ('rf', RandomForestClassifier(n_estimators=200, max_depth=6, min_samples_leaf=4, random_state=42)),
    ('gb', GradientBoostingClassifier(n_estimators=100, max_depth=3, learning_rate=0.1, random_state=42)),
    ('et', ExtraTreesClassifier(n_estimators=200, max_depth=6, min_samples_leaf=4, random_state=42)),
    ('ada', AdaBoostClassifier(n_estimators=100, learning_rate=0.5, random_state=42)),
    ('svc', SVC(kernel='rbf', C=1.0, probability=True, random_state=42)),
    ('xgb', XGBClassifier(n_estimators=100, max_depth=3, learning_rate=0.1, 
                          use_label_encoder=False, eval_metric='logloss', random_state=42))  # Simpler than baseline
]

print("Models in ensemble:")
for name, model in models:
    print(f"  - {name}: {type(model).__name__}")

Models in ensemble:
  - lr: LogisticRegression
  - rf: RandomForestClassifier
  - gb: GradientBoostingClassifier
  - et: ExtraTreesClassifier
  - ada: AdaBoostClassifier
  - svc: SVC
  - xgb: XGBClassifier


In [6]:
# 10-fold Stratified Cross-Validation with Voting Ensemble
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

cv_scores = []
oof_predictions = np.zeros(len(X))
test_predictions = np.zeros(len(X_test))

# Also track individual model scores for analysis
individual_scores = {name: [] for name, _ in models}

for fold, (train_idx, val_idx) in enumerate(kfold.split(X, y)):
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    # Scale features for SVC and LogisticRegression
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)
    
    # Train individual models and collect predictions for soft voting
    fold_probs = np.zeros((len(X_val), 2))
    test_fold_probs = np.zeros((len(X_test), 2))
    
    for name, model in models:
        # Use scaled data for SVC and LR
        if name in ['lr', 'svc']:
            model.fit(X_train_scaled, y_train)
            val_prob = model.predict_proba(X_val_scaled)
            test_prob = model.predict_proba(X_test_scaled)
            val_pred = model.predict(X_val_scaled)
        else:
            model.fit(X_train, y_train)
            val_prob = model.predict_proba(X_val)
            test_prob = model.predict_proba(X_test)
            val_pred = model.predict(X_val)
        
        fold_probs += val_prob
        test_fold_probs += test_prob
        
        # Track individual model accuracy
        individual_scores[name].append(accuracy_score(y_val, val_pred))
    
    # Average probabilities (soft voting)
    fold_probs /= len(models)
    test_fold_probs /= len(models)
    
    # Ensemble predictions
    val_pred_ensemble = (fold_probs[:, 1] >= 0.5).astype(int)
    oof_predictions[val_idx] = val_pred_ensemble
    
    # Accumulate test predictions
    test_predictions += test_fold_probs[:, 1] / kfold.n_splits
    
    # Calculate fold accuracy
    fold_acc = accuracy_score(y_val, val_pred_ensemble)
    cv_scores.append(fold_acc)
    print(f"Fold {fold+1}: Accuracy = {fold_acc:.4f}")

print(f"\n{'='*50}")
print(f"Mean CV Accuracy: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores):.4f})")
print(f"Overall OOF Accuracy: {accuracy_score(y, oof_predictions):.4f}")

Fold 1: Accuracy = 0.8778


Fold 2: Accuracy = 0.7978


Fold 3: Accuracy = 0.8315


Fold 4: Accuracy = 0.8539


Fold 5: Accuracy = 0.8090


Fold 6: Accuracy = 0.8539


Fold 7: Accuracy = 0.8539


Fold 8: Accuracy = 0.8090


Fold 9: Accuracy = 0.8427


Fold 10: Accuracy = 0.8427

Mean CV Accuracy: 0.8372 (+/- 0.0239)
Overall OOF Accuracy: 0.8373


In [7]:
# Individual model performance analysis
print("\nIndividual Model Performance:")
print("="*50)
for name, scores in individual_scores.items():
    print(f"{name:5s}: {np.mean(scores):.4f} (+/- {np.std(scores):.4f})")

print(f"\nEnsemble: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores):.4f})")
print(f"\nBaseline (XGBoost alone): 0.8316 (+/- 0.0324)")
print(f"Improvement: {np.mean(cv_scores) - 0.8316:+.4f}")


Individual Model Performance:
lr   : 0.8136 (+/- 0.0329)
rf   : 0.8282 (+/- 0.0204)
gb   : 0.8260 (+/- 0.0211)
et   : 0.8316 (+/- 0.0216)
ada  : 0.8193 (+/- 0.0279)
svc  : 0.8282 (+/- 0.0249)
xgb  : 0.8260 (+/- 0.0288)

Ensemble: 0.8372 (+/- 0.0239)

Baseline (XGBoost alone): 0.8316 (+/- 0.0324)
Improvement: +0.0056


In [8]:
# Generate submission
test_pred_binary = (test_predictions >= 0.5).astype(int)

submission = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived': test_pred_binary
})

submission.to_csv('/home/submission/submission.csv', index=False)
print(f"Submission saved to /home/submission/submission.csv")
print(f"\nSubmission preview:")
print(submission.head(10))
print(f"\nPrediction distribution:")
print(submission['Survived'].value_counts())
print(f"\nComparison with baseline:")
print(f"Baseline: 267 died, 151 survived")
print(f"Ensemble: {(test_pred_binary == 0).sum()} died, {(test_pred_binary == 1).sum()} survived")

Submission saved to /home/submission/submission.csv

Submission preview:
   PassengerId  Survived
0          892         0
1          893         0
2          894         0
3          895         0
4          896         1
5          897         0
6          898         1
7          899         0
8          900         1
9          901         0

Prediction distribution:
Survived
0    255
1    163
Name: count, dtype: int64

Comparison with baseline:
Baseline: 267 died, 151 survived
Ensemble: 255 died, 163 survived
